In [ ]:
import pandas as pd
import json
import random
from tqdm import tqdm
from itertools import combinations

key="OPENAI_KEY"

from openai import OpenAI
client = OpenAI(api_key=key)

In [3]:
data = pd.read_csv("../data/educhat-translation/all_translations.csv")

In [4]:
entries = []
for i in tqdm(range(len(data))):
    entry = {
        "original" : data["english"].iloc[i],
        "khmer" : data["khmer"].iloc[i]
    }
    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0.9,
        messages=[
            {"role": "system", "content": "Please translate the following text exactly and literally from Khmer to English. Do not summarize or omit details. Preserve all names, terms, and structure as closely as possible. You will output in the following JSON format. {'translated': translated_text}"},
            {
                "role": "user",
                "content": data["khmer"].iloc[i]
            }
        ],
        response_format={"type":"json_object"}
    )
    
    entry["translated"] = json.loads(response.choices[0].message.content)["translated"]
    entries.append(entry)

100%|██████████| 60/60 [03:19<00:00,  3.32s/it]


In [5]:
pd.DataFrame(entries).to_csv("../results/back-translated.csv", index=False)